In [1]:
import pandas as pd
import numpy as np
import re


In [2]:
df = pd.read_csv("data/hotel_csv_data.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 61 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   101                     6665 non-null   int64  
 1   NK                      6665 non-null   object 
 2   1                       6665 non-null   int64  
 3   NUID065253359           6665 non-null   object 
 4   1.1                     6665 non-null   object 
 5   071623 113711 101       6665 non-null   object 
 6   11:37 AM   07/16/2023   6665 non-null   object 
 7   2                       6663 non-null   object 
 8   1.2                     6663 non-null   float64
 9   01:37 PM   07/16/2023   6665 non-null   object 
 10  CreditCard              6665 non-null   object 
 11  F                       6665 non-null   object 
 12  41.08                   6665 non-null   float64
 13  Unnamed: 13             140 non-null    object 
 14  0.00                    6665 non-null   

In [4]:
df.sample(n=5)

,101,NK,1,NUID065253359,1.1,071623 113711 101,11:37 AM 07/16/2023,2,1.2,01:37 PM 07/16/2023,...,41.08.4,0.00.4,0.00.5,Unnamed: 54,F.2,F.3,Unnamed: 57,3,7.00%6.00%4.000%17.000,0000
5635,107,K,1,TX29376837,1,092023 054802 107,05:48 PM 09/20/2023,1,1.0,06:48 PM 09/20/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,12,7.00%6.00%4.000%17.000,0
4064,115,K,1,TX23890054,1,051123 083147 115,08:31 PM 05/11/2023,1,1.0,09:31 PM 05/11/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,7,7.00%6.00%4.000%17.000,0
3918,111,K,1,TX17113940,1,112522 110000 111,11:00 AM 11/28/2022,N,3.0,11:00 AM 12/01/2022,...,120.0,0.0,0.0,NaN,F,F,NaN,5,7.00%6.00%4.000%17.000,0
10,114,K,1,TX40887263,1,071623 110000 114,11:00 AM 07/16/2023,N,1.0,11:00 AM 07/17/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,35,7.00%6.00%4.000%17.000,0
3077,108,K,1,TX44225771,1,102222 113325 108,11:33 PM 10/22/2022,N,1.0,11:00 AM 10/23/2022,...,0.0,0.0,0.0,NaN,F,F,NaN,2,7.00%6.00%4.000%17.000,0


In [5]:
idx_to_keep = [0, 1, 3, 6, 7, 9, 12, 21, 23, 29]

In [6]:
# create new dataframe with specified column indices
df1 = df.iloc[:, idx_to_keep].copy()

In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   101                    6665 non-null   int64  
 1   NK                     6665 non-null   object 
 2   NUID065253359          6665 non-null   object 
 3   11:37 AM   07/16/2023  6665 non-null   object 
 4   2                      6663 non-null   object 
 5   01:37 PM   07/16/2023  6665 non-null   object 
 6   41.08                  6665 non-null   float64
 7   35.11                  6665 non-null   float64
 8   5.97                   6665 non-null   float64
 9   BRANDON BANOS          6665 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 520.8+ KB


In [8]:
# Create a dictionary to rename the columns
column_names = [
    "room_num",
    "room_type",
    "customer_id",
    "checkin_timestamp",
    "stay_duration",
    "checkout_timestamp",
    "total_price",
    "price",
    "tax",
    "customer_name",
]

# Assign column names to the DataFrame
df1.columns = column_names

In [9]:
df1.sample(n=5)

,room_num,room_type,customer_id,checkin_timestamp,stay_duration,checkout_timestamp,total_price,price,tax,customer_name
3327,130,K,NUID45645848/4,12/27/2022 9:46:00 PM,N,12/28/2022 11:00:00 AM,46.33,39.60,6.73,ARMANDO GERRARDO
4325,126,NK,NUIDG612368,01:06 AM 07/24/2023,N,11:00 AM 07/24/2023,70.00,59.83,10.17,ELKA GARCIA
1111,109,K,NUID013979894,03:38 PM 11/09/2022,2,05:38 PM 11/09/2022,30.00,25.64,4.36,EDWIN QUINTANILLA
5941,103,NQQ,TX09608904,11:00 AM 01/26/2023,N,11:00 AM 01/27/2023,70.00,59.83,10.17,BERNICE JONES KELLY
2250,103,NQQ,NUID205407379,11:00 AM 02/25/2023,N,11:00 AM 02/27/2023,0.00,0.00,0.00,EDGAR CAESAR MATACILLAS HERNANDEZ


In [10]:
# Convert checkin_timestamp and checkout_timestamp
def convert_string_to_datetime(df, column_name, new_column_name):
    # Create a datetime object from the string object.
    datetime_object = pd.to_datetime(
        df[column_name], errors="coerce", format="mixed")

    # Add the datetime object to the DataFrame as a new column.
    new_datetime_column = f"{new_column_name}_datetime"
    df[new_datetime_column] = datetime_object

    # drop old timestamp columns
    df = df.drop([column_name], axis=1)

    return df

In [11]:
df1 = convert_string_to_datetime(df1.copy(), "checkin_timestamp", "checkin")
df1 = convert_string_to_datetime(df1.copy(), "checkout_timestamp", "checkout")

In [12]:
# drop old timestamp columns
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   room_num           6665 non-null   int64         
 1   room_type          6665 non-null   object        
 2   customer_id        6665 non-null   object        
 3   stay_duration      6663 non-null   object        
 4   total_price        6665 non-null   float64       
 5   price              6665 non-null   float64       
 6   tax                6665 non-null   float64       
 7   customer_name      6665 non-null   object        
 8   checkin_datetime   6665 non-null   datetime64[ns]
 9   checkout_datetime  6665 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int64(1), object(4)
memory usage: 520.8+ KB


In [13]:
# Convert the column to a categorical type
def convert_dtypes(df, column_name, data_type):
    df[column_name] = df[column_name].astype(data_type)

    return df

In [14]:
# convert object dtypes to category dtype and room_num to category dtype
category_features = ["room_num", "room_type", "stay_duration"]

data_type = "category"

for category in category_features:
    df1 = convert_dtypes(df1.copy(), category, data_type)

In [15]:
# Subtracts two datetime columns in a Pandas DataFrame and calculates the difference into a day and hours column.
def subtract_datetime_columns(df, column_1, column_2):
    # Subtract the two datetime columns.
    # Compare the two datetime columns.
    greater_datetime_column = df[column_1].where(
        df[column_1] > df[column_2], df[column_2]
    )
    least_datetime_column = df[column_1].where(
        df[column_1] <= df[column_2], df[column_2]
    )

    # Subtract the greater datetime column by the least datetime column.
    difference = greater_datetime_column - least_datetime_column
    # difference = df[time1] - df[time0]

    # Convert the difference to a timedelta object.
    difference_timedelta = difference.dt.total_seconds()

    # Calculate the day and hours difference.
    day_difference = difference_timedelta // (24 * 60 * 60)
    hours_difference = (difference_timedelta % (24 * 60 * 60)) // (60 * 60)

    # Add the day and hours difference columns to the DataFrame.
    df["days"] = day_difference
    df["hours"] = hours_difference

    return df

In [16]:
df1 = subtract_datetime_columns(
    df1.copy(), "checkin_datetime", "checkout_datetime")

df1.sample(n=5)

,room_num,room_type,customer_id,stay_duration,total_price,price,tax,customer_name,checkin_datetime,checkout_datetime,days,hours
6315,108,K,TX12220802,N,0.0,0.00,0.00,ALVARADO PAUL EDWARD,2023-02-06 11:00:00,2023-02-07 11:00:00,1.0,0.0
1160,114,K,NUIDMATJ81103HNLLRN04,N,75.0,64.10,10.90,JUAN CARLOS MALDONADO,2023-08-11 11:00:00,2023-08-12 11:00:00,1.0,0.0
1394,118,NQQ,TX29501795,N,75.0,64.10,10.90,KNOBEL LAUREN,2023-09-12 11:00:00,2023-09-13 11:00:00,1.0,0.0
1260,108,K,NUIDN04810684,1,30.0,25.64,4.36,GUZMAN JOCELIN,2023-08-06 02:24:00,2023-08-06 03:24:00,0.0,1.0
1523,109,K,TX18266335,1,25.0,21.37,3.63,GONE CARLOS,2022-12-17 15:35:00,2022-12-17 16:35:00,0.0,1.0


In [17]:
# Bucket checkin_datetime into four time periods.
def bucket_time_period(datetime_value):
    """
    Buckets a datetime value into one of four time periods.
        * Morning (8:00 AM to 12:00 PM)
        * Afternoon (12:00 PM to 5:00 PM)
        * Evening (5:00 PM to 9:00 PM)
        * Night (9:00 PM to 8:00 AM the next day)
    """

    hour = datetime_value.hour

    if hour >= 8 and hour < 12:
        time_period = 'Morning'
    elif hour >= 12 and hour < 17:
        time_period = 'Afternoon'
    elif hour >= 17 and hour < 21:
        time_period = 'Evening'
    else:
        time_period = 'Night'

    return time_period

# parse datetime columns into month, day, and year
def parse_datetime_column(df, datetime_column, datetime_category):
    # Parse the `datetime` column and extract the `month`, `day`, and `year`.
    year = f"{datetime_category}_year"
    month = f"{datetime_category}_month"
    day = f"{datetime_category}_day"
    time = f"{datetime_category}_time"
    weekday = f"{datetime_category}_weekday"
    day_name = f"{datetime_category}_day_name"
    time_period = f"{datetime_category}_time_period"

    df[month] = df[datetime_column].dt.month
    df[day] = df[datetime_column].dt.day
    df[year] = df[datetime_column].dt.year
    df[time] = df[datetime_column].dt.time
    df[weekday] = df[datetime_column].dt.weekday # Monday is 0 and Sunday is 6
    df[day_name] = df[datetime_column].dt.day_name()
    df[time_period] = df[datetime_column].apply(lambda x: bucket_time_period(x))

    return df

In [18]:
df1 = parse_datetime_column(df1.copy(), "checkin_datetime", "checkin")
df1 = parse_datetime_column(df1.copy(), "checkout_datetime", "checkout")

In [19]:
df1.sample()

,room_num,room_type,customer_id,stay_duration,total_price,price,tax,customer_name,checkin_datetime,checkout_datetime,...,checkin_weekday,checkin_day_name,checkin_time_period,checkout_month,checkout_day,checkout_year,checkout_time,checkout_weekday,checkout_day_name,checkout_time_period
2724,105,NKH,TX48470687,2,35.0,29.91,5.09,AREVALO ACEVEDO GERARDO,2023-08-24 12:58:00,2023-08-24 14:58:00,...,3,Thursday,Afternoon,8,24,2023,14:58:00,3,Thursday,Afternoon


In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   room_num              6665 non-null   category      
 1   room_type             6665 non-null   category      
 2   customer_id           6665 non-null   object        
 3   stay_duration         6663 non-null   category      
 4   total_price           6665 non-null   float64       
 5   price                 6665 non-null   float64       
 6   tax                   6665 non-null   float64       
 7   customer_name         6665 non-null   object        
 8   checkin_datetime      6665 non-null   datetime64[ns]
 9   checkout_datetime     6665 non-null   datetime64[ns]
 10  days                  6665 non-null   float64       
 11  hours                 6665 non-null   float64       
 12  checkin_month         6665 non-null   int32         
 13  checkin_day       

In [32]:
# df1.to_csv("/Users/zm00g/Data_Science/regency_suites/data/qinn_processed_data.csv", index=False)

# EDA

## Missing Data Analysis


In [21]:
# Filter the DataFrame to only include rows where the `total_price` column equals 0.0.
price_filtered_df = df1[df1["total_price"] == 0.0].sort_values(
    by="checkin_datetime", ascending=True
)

price_filtered_df

,room_num,room_type,customer_id,stay_duration,total_price,price,tax,customer_name,checkin_datetime,checkout_datetime,...,checkin_weekday,checkin_day_name,checkin_time_period,checkout_month,checkout_day,checkout_year,checkout_time,checkout_weekday,checkout_day_name,checkout_time_period
3053,105,NKH,TX36767843,3,0.0,0.0,0.0,SMITH DOMINIQUE,2022-10-25 13:08:00,2022-10-25 16:08:00,...,1,Tuesday,Afternoon,10,25,2022,16:08:00,1,Tuesday,Afternoon
3055,104,NK,NUID205439626,2,0.0,0.0,0.0,CESAR TORRES,2022-10-25 15:59:00,2022-10-25 17:59:00,...,1,Tuesday,Afternoon,10,25,2022,17:59:00,1,Tuesday,Evening
3060,122,NK,TX20958699,N,0.0,0.0,0.0,MEREDITH RICHARD,2022-10-25 17:22:00,2022-10-26 11:00:00,...,1,Tuesday,Evening,10,26,2022,11:00:00,2,Wednesday,Morning
3063,107,K,NUIDE225062,3,0.0,0.0,0.0,RENE LOPEZ,2022-10-25 19:04:00,2022-10-25 22:04:00,...,1,Tuesday,Evening,10,25,2022,22:04:00,1,Tuesday,Night
6437,116,K,TX16285729,W,0.0,0.0,0.0,GREOGORY BATES,2022-10-26 11:00:00,2022-11-02 11:00:00,...,2,Wednesday,Morning,11,2,2022,11:00:00,2,Wednesday,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5813,105,NKH,TX19205244,N,0.0,0.0,0.0,RAMIREZ LAURA,2023-07-22 01:14:00,2023-07-22 11:00:00,...,5,Saturday,Night,7,22,2023,11:00:00,5,Saturday,Morning
6145,107,K,TX08922944,2,0.0,0.0,0.0,JONES JULIAN SR,2023-08-02 16:38:00,2023-08-02 18:38:00,...,2,Wednesday,Afternoon,8,2,2023,18:38:00,2,Wednesday,Evening
5663,122,NK,LA009138661,N,0.0,0.0,0.0,WRIGHT TYVON MARSE,2023-08-18 13:33:00,2023-08-19 11:00:00,...,4,Friday,Afternoon,8,19,2023,11:00:00,5,Saturday,Morning
5420,124,NK,TX46628099,N,0.0,0.0,0.0,BRANSOM BONNIE,2023-08-22 19:05:00,2023-08-24 11:00:00,...,1,Tuesday,Evening,8,24,2023,11:00:00,3,Thursday,Morning


In [30]:
# price_filtered_df.to_csv("/Users/zm00g/Data_Science/regency_suites/data/missing_price.csv", index=False)

In [22]:
min_date = price_filtered_df["checkin_datetime"].min()
max_date = price_filtered_df["checkout_datetime"].max()

print(f"There are zeroed records spanning from {min_date} to {max_date}")

There are zeroed records spanning from 2022-10-25 13:08:00 to 2023-09-27 11:00:00


In [23]:
# Group the filtered DataFrame by the category column and count the number of rows in each group.
price_filtered_df.groupby("checkin_year")["checkin_month"].value_counts()

checkin_year  checkin_month
2022          12                75
              11                41
              10                 9
2023          4                106
              1                105
              5                105
              3                 82
              2                 67
              6                  7
              7                  7
              8                  3
              9                  1
Name: count, dtype: int64

In [24]:
kishivam_timeperiod = price_filtered_df[
    (price_filtered_df["checkin_month"] >= 6)
    & (price_filtered_df["checkin_month"] <= 9)
    & (price_filtered_df["checkin_year"] == 2023)
    ].sort_values(by="checkin_datetime", ascending=True)

kishivam_timeperiod

,room_num,room_type,customer_id,stay_duration,total_price,price,tax,customer_name,checkin_datetime,checkout_datetime,...,checkin_weekday,checkin_day_name,checkin_time_period,checkout_month,checkout_day,checkout_year,checkout_time,checkout_weekday,checkout_day_name,checkout_time_period
518,117,K,TX%TXHOUST,NaN,0.0,0.0,0.0,ON^RAMIREZ$JOSE$JR^2117 MADGE ST^?;63601510322...,2023-06-15 16:34:00,2023-06-15 16:34:00,...,3,Thursday,Afternoon,6,15,2023,16:34:00,3,Thursday,Afternoon
5765,108,K,TX29088150,D,0.0,0.0,0.0,HARRISON LATOYA,2023-06-21 03:43:00,2023-06-21 08:43:00,...,2,Wednesday,Night,6,21,2023,08:43:00,2,Wednesday,Morning
2102,114,K,TX40887263,N,0.0,0.0,0.0,MYERS JASON JR,2023-06-21 16:00:00,2023-06-22 11:00:00,...,2,Wednesday,Afternoon,6,22,2023,11:00:00,3,Thursday,Morning
4568,104,NK,TX43265970,D,0.0,0.0,0.0,OGWUCHE JOSEPH,2023-06-22 14:58:00,2023-06-22 19:58:00,...,3,Thursday,Afternoon,6,22,2023,19:58:00,3,Thursday,Evening
2960,112,K,TX41901499,N,0.0,0.0,0.0,RODNEY SHANE JILES,2023-06-28 11:00:00,2023-06-29 11:00:00,...,2,Wednesday,Morning,6,29,2023,11:00:00,3,Thursday,Morning
4259,112,K,TX41901499,N,0.0,0.0,0.0,RODNEY SHANE JILES,2023-06-29 11:00:00,2023-06-30 11:00:00,...,3,Thursday,Morning,6,30,2023,11:00:00,4,Friday,Morning
322,112,K,TX41901499,N,0.0,0.0,0.0,RODNEY SHANE JILES,2023-06-30 11:00:00,2023-07-01 11:00:00,...,4,Friday,Morning,7,1,2023,11:00:00,5,Saturday,Morning
328,112,K,TX41901499,N,0.0,0.0,0.0,RODNEY SHANE JILES,2023-07-01 11:00:00,2023-07-07 11:00:00,...,5,Saturday,Morning,7,7,2023,11:00:00,4,Friday,Morning
1855,126,NK,TX41901499,W,0.0,0.0,0.0,RODNEY SHANE JILES,2023-07-09 11:00:00,2023-07-16 11:00:00,...,6,Sunday,Morning,7,16,2023,11:00:00,6,Sunday,Morning
3837,115,K,TX16699939,N,0.0,0.0,0.0,WILLIAMS JOSEPH DENISAL,2023-07-12 01:51:00,2023-07-12 11:00:00,...,2,Wednesday,Night,7,12,2023,11:00:00,2,Wednesday,Morning


In [31]:
# kishivam_timeperiod.to_csv("/Users/zm00g/Data_Science/regency_suites/data/kishivam_timeperiod_missing_price.csv", index=False)

In [25]:
kishivam_timeperiod.groupby("checkin_year")["checkin_month"].value_counts()

checkin_year  checkin_month
2023          6                7
              7                7
              8                3
              9                1
Name: count, dtype: int64

In [26]:
known_zeroes = kishivam_timeperiod[
    (kishivam_timeperiod["customer_name"] == "RODNEY SHANE JILES")
]

known_zeroes

,room_num,room_type,customer_id,stay_duration,total_price,price,tax,customer_name,checkin_datetime,checkout_datetime,...,checkin_weekday,checkin_day_name,checkin_time_period,checkout_month,checkout_day,checkout_year,checkout_time,checkout_weekday,checkout_day_name,checkout_time_period
2960,112,K,TX41901499,N,0.0,0.0,0.0,RODNEY SHANE JILES,2023-06-28 11:00:00,2023-06-29 11:00:00,...,2,Wednesday,Morning,6,29,2023,11:00:00,3,Thursday,Morning
4259,112,K,TX41901499,N,0.0,0.0,0.0,RODNEY SHANE JILES,2023-06-29 11:00:00,2023-06-30 11:00:00,...,3,Thursday,Morning,6,30,2023,11:00:00,4,Friday,Morning
322,112,K,TX41901499,N,0.0,0.0,0.0,RODNEY SHANE JILES,2023-06-30 11:00:00,2023-07-01 11:00:00,...,4,Friday,Morning,7,1,2023,11:00:00,5,Saturday,Morning
328,112,K,TX41901499,N,0.0,0.0,0.0,RODNEY SHANE JILES,2023-07-01 11:00:00,2023-07-07 11:00:00,...,5,Saturday,Morning,7,7,2023,11:00:00,4,Friday,Morning
1855,126,NK,TX41901499,W,0.0,0.0,0.0,RODNEY SHANE JILES,2023-07-09 11:00:00,2023-07-16 11:00:00,...,6,Sunday,Morning,7,16,2023,11:00:00,6,Sunday,Morning
12,126,NK,TX41901499,N,0.0,0.0,0.0,RODNEY SHANE JILES,2023-07-16 11:00:00,2023-07-17 11:00:00,...,6,Sunday,Morning,7,17,2023,11:00:00,0,Monday,Morning
5107,126,NK,TX41901499,N,0.0,0.0,0.0,RODNEY SHANE JILES,2023-07-17 11:00:00,2023-07-18 11:00:00,...,0,Monday,Morning,7,18,2023,11:00:00,1,Tuesday,Morning


In [27]:
known_zeroes.groupby("checkin_year")["checkin_month"].value_counts()

checkin_year  checkin_month
2023          7                4
              6                3
Name: count, dtype: int64

In [28]:
known_zeroes["days"].sum()

18.0